In [3]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam



In [4]:
# Set the path to dataset directory
data_dir = "C:\\Users\\HP\\Downloads\\Dataset"

In [5]:
# Image data augmentation
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

In [6]:
# Load the data
batch_size = 32
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [7]:
# Function to create a MobileNetV2-based mask classification model
def create_mask_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(3, activation='softmax')(x)  # 3 classes: Incorrect Mask, With Mask, Without Mask
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [8]:
# Create the mask classification model
mask_model = create_mask_model()

9406464/9406464 [==============================] - 8s 1us/step


In [9]:
# Compile the model
mask_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Train the model
epochs = 6 # You can adjust the number of epochs
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

mask_model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

# Save the model weights
mask_model.save_weights("path_to_save_model.h5")

Epoch 1/6
75/75 [==============================] - 263s 3s/step - loss: 0.4884 - accuracy: 0.8258 - val_loss: 1.8332 - val_accuracy: 0.7222
Epoch 2/6
75/75 [==============================] - 319s 4s/step - loss: 0.2582 - accuracy: 0.9150 - val_loss: 1.6934 - val_accuracy: 0.6319
Epoch 3/6
75/75 [==============================] - 279s 4s/step - loss: 0.2033 - accuracy: 0.9250 - val_loss: 6.3169 - val_accuracy: 0.4010
Epoch 4/6
75/75 [==============================] - 254s 3s/step - loss: 0.1903 - accuracy: 0.9342 - val_loss: 4.6098 - val_accuracy: 0.5590
Epoch 5/6
75/75 [==============================] - 273s 4s/step - loss: 0.1575 - accuracy: 0.9488 - val_loss: 5.0380 - val_accuracy: 0.5122
Epoch 6/6
75/75 [==============================] - 272s 4s/step - loss: 0.1418 - accuracy: 0.9550 - val_loss: 2.6457 - val_accuracy: 0.6684


In [11]:
import os

# Get the absolute path of the current directory
current_directory = os.path.abspath(".")

# Check if the model weights file exists in the current directory
model_weights_file = "path_to_save_model.h5"
model_weights_path = os.path.join(current_directory, model_weights_file)

if os.path.exists(model_weights_path):
    print("Model weights are saved at:", model_weights_path)
else:
    print("Model weights file not found.")

Model weights are saved at: C:\Users\HP\Downloads\path_to_save_model.h5


In [12]:
import cv2
import numpy as np
import requests
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

In [13]:
# After training the model
mask_model.save("path_to_save_model.h5")

C:\Users\HP\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:
# Later, to load the model
from tensorflow.keras.models import load_model

loaded_model = load_model("path_to_save_model.h5")

In [15]:
# Load the pre-trained mask classification model (MobileNet)
mask_model = load_model("C:\\Users\\HP\\Downloads\\path_to_save_model.h5")

In [16]:
# Define the labels for mask classification
labels = {0: "Incorrect Mask", 1: "With Mask", 2: "Without Mask"}

In [17]:
# Load the pre-trained face detection model from OpenCV (SSD MobileNet V2)
face_net = cv2.dnn.readNetFromCaffe(
    "E:\\deploy.prototxt.txt",
    "E:\\res10_300x300_ssd_iter_140000.caffemodel"
)

In [18]:
cap = cv2.VideoCapture('http://192.168.0.100:4747/video')

while True:
    # Read the frame from the webcam
    ret, frame = cap.read()

    if not ret:
        break

    # Perform face detection on the frame
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
    face_net.setInput(blob)
    detections = face_net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # Check if the face detection confidence is greater than a threshold
        if confidence > 0.5:
            # Compute the (x, y)-coordinates of the bounding box for the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Extract the face region from the frame
            face = frame[startY:endY, startX:endX]

            if face.shape[0] < 10 or face.shape[1] < 10:
                # Ignore very small faces (might be false detections)
                continue

            # Preprocess the face for mask classification
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)

            # Classify the face using the mask classification model
            prediction = mask_model.predict(face)
            class_id = np.argmax(prediction)
            label = labels[class_id]

            # Draw the label and bounding box on the frame
            label = f"Mask: {label}"
            cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

    # Display the output frame
    cv2.imshow("Mask Detection", frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 31ms/step
